In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
import webbrowser
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

In [2]:
summoner = 'Saikki kusuo'
key = 'RGAPI-9724b32a-f354-408c-8cde-8fdcc35e01fa'

## Extracting Functions

In [3]:
# Summoner v4, /lol/summoner/v4/summoners/by-name/{summonerName}
# Get summoner info by name using API key and summoner name

def get_summoner_info(summoner_name, apikey):
    # This function takes the summoner name and the API Key
    # and retrieves its account id
    url = f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}'
    html = requests.get(url,
                       params={'api_key': apikey})
    json = html.json()
    df = pd.DataFrame(json, index=[0])
    
    return df

In [4]:
def get_matchlist(accountid, api, champions, queues):
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountid}'
    html = requests.get(url,
                        params={'api_key': api})
    json = html.json()
    df = pd.DataFrame(json['matches'])
    df['Date'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime("%d/%m/%Y %A, %H:%M:%S"))
    df['Date2'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime("%Y/%m/%d"))
    df = df.merge(champions, left_on='champion', right_on='champion', how='left')
    df = df.merge(queues, left_on='queue', right_on='queue', how='left')

    df.drop(['champion', 'queue'], axis=1, inplace=True)
    df.rename(columns={'name': 'champion', 'queue_type': 'queue'}, inplace=True)

    return df

In [5]:
def get_match_info(matchid):
    
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matches/{matchid}'
    html = requests.get(url,
                    params={'api_key': key})
    json = html.json()
    
    return json

In [6]:
def get_players_info(json):
        
    players = pd.DataFrame()
    
    for participant in range(10):
        df = pd.DataFrame.from_dict(json['participants'][participant]['stats'], orient='index').T
        df.insert(loc=0, column='championId', value= json['participants'][participant]['championId'])
        df.insert(loc=1, column='teamId', value=json['participants'][participant]['teamId'])
        
        dff = pd.DataFrame.from_dict(json['participantIdentities'][participant]['player'], orient='index').T
        dff.insert(loc=0, column='participantId', value= json['participantIdentities'][participant]['participantId'])
        df_final = pd.merge(df, dff, on='participantId')
        
        players = players.append(df_final)
    
    players.insert(loc=0, column='gameid', value=[json['gameId'] for value in range(len(players))])
    
    return players

In [7]:
def get_match_timeline(matchid):

    url= f'https://euw1.api.riotgames.com/lol/match/v4/timelines/by-match/{matchid}'
    html = requests.get(url,
                       params={'api_key': key})
    json = html.json()
    
    return json

In [8]:
def participant_frames(json, playersinfo):
    ## Extracting frames from json

    all_frames = pd.DataFrame()

    for frame in list(range(len(json['frames']))):
        iterable = json['frames'][frame]['participantFrames']
        for elem in iterable:
            dic = {}
            for sub_elem in iterable[elem]:
                if type(iterable[elem][sub_elem]) == dict:
                    dic.update(iterable[elem][sub_elem])
                else:
                    dic[sub_elem] = iterable[elem][sub_elem]
            df = pd.DataFrame(dic, index=[0])
            df.insert(loc=0, column='timestamp', value=json['frames'][frame]['timestamp'])
            all_frames = all_frames.append(df, ignore_index=True)

    cols = ['timestamp', 'participantId', 'x', 'y', 'currentGold', 'totalGold', 'level',
            'xp', 'minionsKilled', 'jungleMinionsKilled',
            # 'dominionScore','teamScore',
            ]
    all_frames = all_frames[cols]
    
    teams = playersinfo[['participantId','teamId']]
    frames = pd.merge(all_frames, teams, on='participantId')
    frames = pd.merge(frames, players_info[['participantId', 'summonerName', 'championId']], on='participantId')
    frames['timestamp'] = (frames['timestamp'] / 60000).astype('int')

    return frames

In [9]:
def get_events(json):
    
    events = pd.DataFrame()
    
    frames = json['frames']
    for elem in range(len(frames)):
        events = events.append(pd.DataFrame(frames[elem]['events']))
    
    
    events['position_x'] = [elem.get('x') if type(elem) == dict else 'none' for elem in events['position']]
    events['position_y'] = [elem.get('y') if type(elem) == dict else 'none' for elem in events['position']]
    events.drop(columns='position', inplace=True)
    events['timestamp'] = (events['timestamp'] / 60000).astype('int')
    events.drop(columns=['teamId'], axis=1, inplace=True)
    events = events.merge(players_info[['participantId', 'teamId']], on='participantId', how='left')
    events = events.merge(players_info[['participantId', 'teamId']], left_on='killerId', right_on='participantId', how='outer')
    
    return events

In [10]:
def get_championsid(api):
    url = 'http://ddragon.leagueoflegends.com/cdn/10.21.1/data/en_US/champion.json'

    html = requests.get(url,
                        params={'api_key': api})
    json = html.json()

    champions = pd.DataFrame()
    for champ in json['data'].keys():
        df = pd.DataFrame({'name': json['data'][f'{champ}']['id'], 'champion': json['data'][f'{champ}']['key']},
                          index=[0])
        champions = champions.append(df, ignore_index=True)
    champions['champion'] = champions['champion'].astype('int')
    
    return champions

In [11]:
def get_queuesid(api):
    url = 'http://static.developer.riotgames.com/docs/lol/queues.json'
    html = requests.get(url,
                        params={'api_key': api})
    json = html.json()

    queuesid = pd.DataFrame()
    for queue in json:
        df = pd.DataFrame({'queue': queue['queueId'], 'queue_type': queue['description']},
                          index=[0])
        queuesid = queuesid.append(df, ignore_index=True)

    return queuesid

In [12]:
def gold_diff(df):
    data = df.groupby(['timestamp', 'teamId']).sum().reset_index()[['timestamp', 'teamId', 'totalGold']]
    
    team100gold = data[data['teamId'] == 100]
    team100gold.rename(columns={"totalGold": "team100gold"}, inplace=True)
    team200gold = data[data['teamId'] == 200][['timestamp', 'teamId', 'totalGold']]
    team200gold.rename(columns={"totalGold": "team200gold"}, inplace=True)
    
    golddiff = pd.merge(team100gold, team200gold, on='timestamp')
    golddiff['golddiff'] = golddiff['team100gold'] - golddiff['team200gold']
    golddiff['team100golddiff'] = [a if a > 0 else 0 for a in golddiff['golddiff']]
    golddiff['team200golddiff'] = [a if a < 0 else 0 for a in golddiff['golddiff']]
    
    return golddiff

In [13]:
def player_stats_table(frames_df, champions):
    
    df = frames_df.copy()
    df.rename(columns={'championId': 'champion'}, inplace=True)
    df = df.merge(champions, on='champion')
    df.drop(['champion'], axis=1, inplace=True)
    df.rename(columns={'name': 'champion'}, inplace=True)
    df = df[['timestamp', 'summonerName', 'champion', 'level', 'minionsKilled', 'totalGold', 'teamId']]
    
    return df

In [14]:
summoner_info = get_summoner_info(summoner, key)
accountid = summoner_info['accountId'][0]
accountid

'djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c'

In [15]:
summoner_info = get_summoner_info(summoner, key)
summoner_info

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,iqJz7QEm4AJIHuEKecIyvBZ65qaANfGcYM9iYL-S_FT2ddQ,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,9XoYu4fgD-ybcBawfY2XsMsGgvv9Av44llBjAhHFZk-nX5...,Saikki Kusuo,4249,1603715879000,196


In [16]:
champions = get_championsid(key)
queues = get_queuesid(key)

In [17]:
match_list = get_matchlist(accountid, key, champions, queues)
match_list

,platformId,gameId,season,timestamp,role,lane,Date,Date2,champion,queue
0,EUW1,4897505128,13,1604069001001,SOLO,TOP,"30/10/2020 Friday, 15:43:21",2020/10/30,Rumble,5v5 Ranked Solo games
1,EUW1,4897397777,13,1604066389283,SOLO,TOP,"30/10/2020 Friday, 14:59:49",2020/10/30,Shen,5v5 Ranked Solo games
2,EUW1,4897481108,13,1604063990617,DUO_SUPPORT,BOTTOM,"30/10/2020 Friday, 14:19:50",2020/10/30,Pantheon,5v5 Ranked Solo games
3,EUW1,4897315548,13,1604061310612,DUO_CARRY,BOTTOM,"30/10/2020 Friday, 13:35:10",2020/10/30,Vayne,5v5 Ranked Solo games
4,EUW1,4895567374,13,1603986975644,SOLO,TOP,"29/10/2020 Thursday, 16:56:15",2020/10/29,Yasuo,5v5 Ranked Solo games
...,...,...,...,...,...,...,...,...,...,...
95,EUW1,4852948640,13,1601905357003,DUO_CARRY,BOTTOM,"05/10/2020 Monday, 15:42:37",2020/10/05,Samira,5v5 Ranked Solo games
96,EUW1,4852952829,13,1601902637078,SOLO,BOTTOM,"05/10/2020 Monday, 14:57:17",2020/10/05,Ezreal,5v5 Ranked Solo games
97,EUW1,4852774276,13,1601891814800,DUO_CARRY,BOTTOM,"05/10/2020 Monday, 11:56:54",2020/10/05,Kaisa,5v5 Ranked Solo games
98,EUW1,4852688998,13,1601887255733,DUO_CARRY,BOTTOM,"05/10/2020 Monday, 10:40:55",2020/10/05,Ezreal,5v5 Ranked Solo games


In [18]:
pd.DataFrame(match_list.iloc[0])

,0
platformId,EUW1
gameId,4897505128
season,13
timestamp,1604069001001
role,SOLO
lane,TOP
Date,"30/10/2020 Friday, 15:43:21"
Date2,2020/10/30
champion,Rumble
queue,5v5 Ranked Solo games


In [19]:
match_info = get_match_info(4884450178)

In [20]:
players_info = get_players_info(match_info)
players_info

,gameid,championId,teamId,participantId,win,item0,item1,item2,item3,item4,...,statPerk1,statPerk2,platformId,accountId,summonerName,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,4884450178,157,100,1,False,3006,3046,0,1055,1038,...,5008,5003,EUW1,n0JthD6A38yyUb2AuQo9NsCkWjdV73GVK5HXyAKvWbAMIV...,allways mid diff,-oeDL_daEobwDxVyKKeCbwH5rHEirK_VXoV-Q7BbaXg78GVV,EUW1,n0JthD6A38yyUb2AuQo9NsCkWjdV73GVK5HXyAKvWbAMIV...,/v1/stats/player_history/EUW1/2536701398738144,4756
0,4884450178,82,100,2,False,3047,3151,3076,1026,1028,...,5008,5002,EUW1,zDMe-lz4c4qbo32vbkELgYqeAU9Ma7UnASj4LYjK3uz3mt8,Best Yasuo Soria,UNQsDdmiHLx24GwSobFFZRsHCnvGLKMKUYQ7TLwkfdJ3u3A,EUW1,zDMe-lz4c4qbo32vbkELgYqeAU9Ma7UnASj4LYjK3uz3mt8,/v1/stats/player_history/EUW1/225008272,4073
0,4884450178,35,100,3,False,3851,1082,3191,0,1001,...,5008,5002,EUW1,rOb3g-_pNqIvRrYF_wwvBgi23afVfGPfAqx9nG1C7Xhk-g,Tkus,YYz-g5GEbfC5snISMcwi2N5a9zvFadgqI01qskcl3PUcaXM,EUW1,rOb3g-_pNqIvRrYF_wwvBgi23afVfGPfAqx9nG1C7Xhk-g,/v1/stats/player_history/EUW1/22985916,4496
0,4884450178,81,100,4,False,1083,3004,3006,3057,1027,...,5008,5002,EUW1,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,Saikki Kusuo,iqJz7QEm4AJIHuEKecIyvBZ65qaANfGcYM9iYL-S_FT2ddQ,EUW1,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,/v1/stats/player_history/EUW1/229546365,4249
0,4884450178,20,100,5,False,1413,2031,3047,1031,1011,...,5008,5003,EUW1,cty5pdyVeFIwARhEANBomo-BuAd2CL6JZnTCSXephJyhFM...,Assassinè,xU_5DEWsl1BJmnz9kiQ0aJTIDtMYSZ0cjY0KenaKJyxQa-l6,EUW1,cty5pdyVeFIwARhEANBomo-BuAd2CL6JZnTCSXephJyhFM...,/v1/stats/player_history/EUW1/2443650064446336,3182
0,4884450178,104,200,6,True,1412,2031,2055,3086,1001,...,5008,5002,EUW1,gHxASn3R8ShwbKhgnFG-AZbtIQqc7CNuPwuFxbCWG3Cq6W...,x2Joit,ObwyBjaFLRKnyqMbGBv1LNvgrIbXKw-P6WV-qwpdjNcs4fCt,EUW1,gHxASn3R8ShwbKhgnFG-AZbtIQqc7CNuPwuFxbCWG3Cq6W...,/v1/stats/player_history/EUW1/2214365005957632,906
0,4884450178,86,200,7,True,2031,1054,3006,3078,3105,...,5008,5003,EUW1,KVwrTgPnRyCvT2cqtqOiP1_HwnbjgDpJl3T5Cg0xxVuwxr...,dZlUpton,EB740d0V3SvcljYtxF_dxC2PjVrF4HYoGAWSJaF-KIAHbGP-,EUW1,KVwrTgPnRyCvT2cqtqOiP1_HwnbjgDpJl3T5Cg0xxVuwxr...,/v1/stats/player_history/EUW1/2277277454321824,4568
0,4884450178,142,200,8,True,1056,2031,3285,3020,0,...,5008,5002,EUW1,dtYBrS4eoC8REaH6pspoAakIoz6-Ir7HG2iXlme1qBFdW1A,Habibibloxberg24,9_Gm_efyGa08UBexjEGs8i_lJFD3xh2EfZyihNMnytmVrms,EUW1,dtYBrS4eoC8REaH6pspoAakIoz6-Ir7HG2iXlme1qBFdW1A,/v1/stats/player_history/EUW1/223538384,1164
0,4884450178,89,200,9,True,3859,3050,3117,1031,1082,...,5008,5001,EUW1,J2YGWcH_vHeLhpnFo_75j1rWKCcXpJPDqz270S-QOWEDtrQ,HydroNightmare,CJj-3CJy0USMa5MJ8zFDr4-xxjM5QPXzcLRF7rdgBjk2eXg,EUW1,J2YGWcH_vHeLhpnFo_75j1rWKCcXpJPDqz270S-QOWEDtrQ,/v1/stats/player_history/EUW1/222812646,4457
0,4884450178,360,200,10,True,1055,3072,3111,3031,3508,...,5008,5001,EUW1,_M8o4ssb90EMMcOg71Z8hgxtqkwX0K8XbKbT0UYw0ftNXP...,Vos grands mere,doYryfsC855kayOnlQOFs3d0sH19y8hp07dcE6aZPAt6DErA,EUW1,_M8o4ssb90EMMcOg71Z8hgxtqkwX0K8XbKbT0UYw0ftNXP...,/v1/stats/player_history/EUW1/2416047671644576,3583


In [21]:
list(players_info)

['gameid',
 'championId',
 'teamId',
 'participantId',
 'win',
 'item0',
 'item1',
 'item2',
 'item3',
 'item4',
 'item5',
 'item6',
 'kills',
 'deaths',
 'assists',
 'largestKillingSpree',
 'largestMultiKill',
 'killingSprees',
 'longestTimeSpentLiving',
 'doubleKills',
 'tripleKills',
 'quadraKills',
 'pentaKills',
 'unrealKills',
 'totalDamageDealt',
 'magicDamageDealt',
 'physicalDamageDealt',
 'trueDamageDealt',
 'largestCriticalStrike',
 'totalDamageDealtToChampions',
 'magicDamageDealtToChampions',
 'physicalDamageDealtToChampions',
 'trueDamageDealtToChampions',
 'totalHeal',
 'totalUnitsHealed',
 'damageSelfMitigated',
 'damageDealtToObjectives',
 'damageDealtToTurrets',
 'visionScore',
 'timeCCingOthers',
 'totalDamageTaken',
 'magicalDamageTaken',
 'physicalDamageTaken',
 'trueDamageTaken',
 'goldEarned',
 'goldSpent',
 'turretKills',
 'inhibitorKills',
 'totalMinionsKilled',
 'neutralMinionsKilled',
 'neutralMinionsKilledTeamJungle',
 'neutralMinionsKilledEnemyJungle',
 'to

In [22]:
players_info[['summonerName', 'kills', 'deaths']]

,summonerName,kills,deaths
0,allways mid diff,3,6
0,Best Yasuo Soria,3,3
0,Tkus,5,12
0,Saikki Kusuo,3,9
0,Assassinè,3,6
0,x2Joit,2,1
0,dZlUpton,3,4
0,Habibibloxberg24,5,3
0,HydroNightmare,1,6
0,Vos grands mere,25,3


In [23]:
players_info[players_info['summonerName'] == 'Saikki Kusuo']['win'][0]

False

In [24]:
timeline = get_match_timeline(4884450178)

In [25]:
frames = participant_frames(timeline, players_info)
frames

,timestamp,participantId,x,y,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled,teamId,summonerName,championId
0,0,2,560.0,581.0,500,500,1,0,0,0,100,Best Yasuo Soria,82
1,1,2,5187.0,8544.0,0,500,1,0,0,0,100,Best Yasuo Soria,82
2,2,2,1913.0,12183.0,63,563,1,120,2,0,100,Best Yasuo Soria,82
3,3,2,2409.0,13356.0,717,1217,3,856,8,0,100,Best Yasuo Soria,82
4,4,2,2714.0,12656.0,462,1697,3,1091,12,0,100,Best Yasuo Soria,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,16,9,11201.0,3425.0,259,5159,9,5591,20,0,200,HydroNightmare,89
206,17,9,5373.0,5896.0,265,5565,9,6059,22,0,200,HydroNightmare,89
207,18,9,5113.0,5073.0,555,5855,10,6497,22,0,200,HydroNightmare,89
208,19,9,2426.0,1754.0,1083,6383,11,7347,22,0,200,HydroNightmare,89


In [26]:
events = get_events(timeline)
events

,type,timestamp,participantId_x,itemId,skillSlot,levelUpType,afterId,beforeId,wardType,creatorId,...,monsterType,monsterSubType,buildingType,laneType,towerType,position_x,position_y,teamId_x,participantId_y,teamId_y
0,ITEM_PURCHASED,0,2,1054.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
1,SKILL_LEVEL_UP,0,4,NaN,1.0,NORMAL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
2,ITEM_PURCHASED,0,2,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
3,ITEM_PURCHASED,0,2,3340.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
4,ITEM_PURCHASED,0,6,1041.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,200.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,BUILDING_KILL,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,TOWER_BUILDING,TOP_LANE,OUTER_TURRET,981,10441,NaN,7,200.0
507,CHAMPION_KILL,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4674,10063,NaN,7,200.0
508,BUILDING_KILL,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,TOWER_BUILDING,MID_LANE,BASE_TURRET,3651,3696,NaN,7,200.0
509,CHAMPION_KILL,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2104,2471,NaN,7,200.0


In [27]:
events['type'].unique()

array(['ITEM_PURCHASED', 'SKILL_LEVEL_UP', 'ITEM_UNDO', 'WARD_PLACED',
       'ITEM_DESTROYED', 'ITEM_SOLD', 'CHAMPION_KILL', 'WARD_KILL',
       'BUILDING_KILL', 'ELITE_MONSTER_KILL'], dtype=object)

In [28]:
events[events['type'] == 'ITEM_DESTROYED']

,type,timestamp,participantId_x,itemId,skillSlot,levelUpType,afterId,beforeId,wardType,creatorId,...,monsterType,monsterSubType,buildingType,laneType,towerType,position_x,position_y,teamId_x,participantId_y,teamId_y
49,ITEM_DESTROYED,0,6,3340.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,200.0,NaN,NaN
59,ITEM_DESTROYED,2,7,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,200.0,NaN,NaN
60,ITEM_DESTROYED,2,2,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
67,ITEM_DESTROYED,2,10,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,200.0,NaN,NaN
72,ITEM_DESTROYED,2,3,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,ITEM_DESTROYED,18,3,1052.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
415,ITEM_DESTROYED,18,8,2055.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,200.0,NaN,NaN
419,ITEM_DESTROYED,18,4,1027.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
421,ITEM_DESTROYED,18,1,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN


In [29]:
golddiff = gold_diff(frames)

/home/david/miniconda3/envs/ironhack/lib/python3.7/site-packages/pandas/core/frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
golddiff

,timestamp,teamId_x,team100gold,teamId_y,team200gold,golddiff,team100golddiff,team200golddiff
0,0,100,2500,200,2500,0,0,0
1,1,100,2500,200,2500,0,0,0
2,2,100,2954,200,2989,-35,0,-35
3,3,100,4787,200,4225,562,562,0
4,4,100,7403,200,5836,1567,1567,0
5,5,100,9129,200,7269,1860,1860,0
6,6,100,10523,200,8842,1681,1681,0
7,7,100,11597,200,11367,230,230,0
8,8,100,13139,200,13236,-97,0,-97
9,9,100,14611,200,15715,-1104,0,-1104


In [31]:
players_info['championId']

0    157
0     82
0     35
0     81
0     20
0    104
0     86
0    142
0     89
0    360
Name: championId, dtype: int64

In [32]:
frames

,timestamp,participantId,x,y,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled,teamId,summonerName,championId
0,0,2,560.0,581.0,500,500,1,0,0,0,100,Best Yasuo Soria,82
1,1,2,5187.0,8544.0,0,500,1,0,0,0,100,Best Yasuo Soria,82
2,2,2,1913.0,12183.0,63,563,1,120,2,0,100,Best Yasuo Soria,82
3,3,2,2409.0,13356.0,717,1217,3,856,8,0,100,Best Yasuo Soria,82
4,4,2,2714.0,12656.0,462,1697,3,1091,12,0,100,Best Yasuo Soria,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,16,9,11201.0,3425.0,259,5159,9,5591,20,0,200,HydroNightmare,89
206,17,9,5373.0,5896.0,265,5565,9,6059,22,0,200,HydroNightmare,89
207,18,9,5113.0,5073.0,555,5855,10,6497,22,0,200,HydroNightmare,89
208,19,9,2426.0,1754.0,1083,6383,11,7347,22,0,200,HydroNightmare,89


In [33]:
player_stats = player_stats_table(frames, champions)
player_stats

,timestamp,summonerName,champion,level,minionsKilled,totalGold,teamId
0,0,Best Yasuo Soria,Mordekaiser,1,0,500,100
1,1,Best Yasuo Soria,Mordekaiser,1,0,500,100
2,2,Best Yasuo Soria,Mordekaiser,1,2,563,100
3,3,Best Yasuo Soria,Mordekaiser,3,8,1217,100
4,4,Best Yasuo Soria,Mordekaiser,3,12,1697,100
...,...,...,...,...,...,...,...
205,16,HydroNightmare,Leona,9,20,5159,200
206,17,HydroNightmare,Leona,9,22,5565,200
207,18,HydroNightmare,Leona,10,22,5855,200
208,19,HydroNightmare,Leona,11,22,6383,200


In [34]:
events

,type,timestamp,participantId_x,itemId,skillSlot,levelUpType,afterId,beforeId,wardType,creatorId,...,monsterType,monsterSubType,buildingType,laneType,towerType,position_x,position_y,teamId_x,participantId_y,teamId_y
0,ITEM_PURCHASED,0,2,1054.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
1,SKILL_LEVEL_UP,0,4,NaN,1.0,NORMAL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
2,ITEM_PURCHASED,0,2,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
3,ITEM_PURCHASED,0,2,3340.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
4,ITEM_PURCHASED,0,6,1041.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,200.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,BUILDING_KILL,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,TOWER_BUILDING,TOP_LANE,OUTER_TURRET,981,10441,NaN,7,200.0
507,CHAMPION_KILL,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4674,10063,NaN,7,200.0
508,BUILDING_KILL,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,TOWER_BUILDING,MID_LANE,BASE_TURRET,3651,3696,NaN,7,200.0
509,CHAMPION_KILL,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2104,2471,NaN,7,200.0


In [35]:
events.iloc[:3]

,type,timestamp,participantId_x,itemId,skillSlot,levelUpType,afterId,beforeId,wardType,creatorId,...,monsterType,monsterSubType,buildingType,laneType,towerType,position_x,position_y,teamId_x,participantId_y,teamId_y
0,ITEM_PURCHASED,0,2,1054.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
1,SKILL_LEVEL_UP,0,4,NaN,1.0,NORMAL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN
2,ITEM_PURCHASED,0,2,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,none,100.0,NaN,NaN


In [43]:
match_list[match_list['Date2'] == 2020/10/30]

,platformId,gameId,season,timestamp,role,lane,Date,Date2,champion,queue


In [46]:
a = "2020-10-30"
a.replace("-", "/")

'2020/10/30'